In [5]:
!pip install joblib



In [3]:
import joblib


In [5]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import mode

  # for scikit-learn models

# Load data for newdata.csv
nd_ch1 = []
with open("newdata.csv", "r") as data:
    for line in data:
        # Format the line
        line = line.split(",")         # Convert to a list
        nd_ch1.append(float(line[0]))  # Just record the 1st channel

nd_ch2 = []
with open("newdata.csv", "r") as data:
    for line in data:
        # Format the line
        line = line.split(",")         # Convert to a list
        nd_ch2.append(float(line[1]))  # Just record the 1st channel

nd_ch3 = []
with open("newdata.csv", "r") as data:
    for line in data:
        # Format the line
        line = line.split(",")         # Convert to a list
        nd_ch3.append(float(line[2]))  # Just record the 1st channel

nd_ch4 = []
with open("newdata.csv", "r") as data:
    for line in data:
        # Format the line
        line = line.split(",")         # Convert to a list
        nd_ch4.append(float(line[3]))  # Just record the 1st channel

# CHANNEL 1
# ============================================

# Assuming nd_ch1_ contains your channel data
nd_ch1_fft = np.fft.fft(nd_ch1)
nd_ch1_freq = np.fft.fftfreq(len(nd_ch1)) * 256

# Apply Fourier Transform
nd_ch1_fft = np.fft.fft(nd_ch1)
nd_ch1_freq = np.fft.fftfreq(len(nd_ch1)) * 256

# Define a threshold frequency
threshold_freq = 100

# Remove frequencies greater than the threshold
nd_ch1_fft[np.abs(nd_ch1_freq) > threshold_freq] = 0

# Remove powerline interference
cut_freq = 60
tolerance = 2
index_to_remove = np.where((nd_ch1_freq >= cut_freq - tolerance) & (nd_ch1_freq <= cut_freq + tolerance))
nd_ch1_fft[index_to_remove] = 0

# binning function
def computeBands(freq_array, filtered_data):
    bandTotals = [0, 0, 0, 0, 0]
    bandCounts = [0, 0, 0, 0, 0]

    for point in range(len(freq_array)):
        magnitude = np.abs(filtered_data[point])
        if freq_array[point] < 4 and magnitude > 0:
            bandTotals[0] += magnitude
            bandCounts[0] += 1
        elif freq_array[point] < 8 and magnitude > 0:
            bandTotals[1] += magnitude
            bandCounts[1] += 1
        elif freq_array[point] < 12 and magnitude > 0:
            bandTotals[2] += magnitude
            bandCounts[2] += 1
        elif freq_array[point] < 30 and magnitude > 0:
            bandTotals[3] += magnitude
            bandCounts[3] += 1
        elif freq_array[point] < 100 and magnitude > 0:
            bandTotals[4] += magnitude
            bandCounts[4] += 1

    # Calculate the average of all points
    bandAverages = np.array(bandTotals) / np.array(bandCounts)
    return bandAverages

# Example usage for Channel 1
nd_ch1_bands_before_ifft = computeBands(nd_ch1_freq, nd_ch1_fft)

# CHANNEL 2
# ============================================

# Assuming nd_ch1_ contains your channel data
nd_ch2_fft = np.fft.fft(nd_ch2)
nd_ch2_freq = np.fft.fftfreq(len(nd_ch2)) * 256

# Apply Fourier Transform
nd_ch2_fft = np.fft.fft(nd_ch2)
nd_ch2_freq = np.fft.fftfreq(len(nd_ch2)) * 256

# Define a threshold frequency
threshold_freq = 100

# Remove frequencies greater than the threshold
nd_ch2_fft[np.abs(nd_ch2_freq) > threshold_freq] = 0

# Remove powerline interference
cut_freq = 60
tolerance = 2
index_to_remove = np.where((nd_ch2_freq >= cut_freq - tolerance) & (nd_ch2_freq <= cut_freq + tolerance))
nd_ch2_fft[index_to_remove] = 0

# CHANNEL 3
# ============================================

# Assuming nd_ch1_ contains your channel data
nd_ch3_fft = np.fft.fft(nd_ch3)
nd_ch3_freq = np.fft.fftfreq(len(nd_ch3)) * 256

# Apply Fourier Transform
nd_ch3_fft = np.fft.fft(nd_ch3)
nd_ch3_freq = np.fft.fftfreq(len(nd_ch3)) * 256

# Define a threshold frequency
threshold_freq = 100

# Remove frequencies greater than the threshold
nd_ch3_fft[np.abs(nd_ch3_freq) > threshold_freq] = 0

# Remove powerline interference
cut_freq = 60
tolerance = 2
index_to_remove = np.where((nd_ch3_freq >= cut_freq - tolerance) & (nd_ch3_freq <= cut_freq + tolerance))
nd_ch3_fft[index_to_remove] = 0

# CHANNEL 4
# ============================================

# Assuming nd_ch1_ contains your channel data
nd_ch4_fft = np.fft.fft(nd_ch4)
nd_ch4_freq = np.fft.fftfreq(len(nd_ch4)) * 256

# Apply Fourier Transform
nd_ch4_fft = np.fft.fft(nd_ch4)
nd_ch4_freq = np.fft.fftfreq(len(nd_ch4)) * 256

# Combine FFT data into a dataframe
nd_df = pd.DataFrame({
    'Ch1': nd_ch1_fft,
    'Ch2': nd_ch2_fft,
    'Ch3': nd_ch3_fft,
    'Ch4': nd_ch4_fft
})

df = nd_df

import numpy as np

# Assuming df is your DataFrame
df['Ch1'] = np.abs(df['Ch1'])
df['Ch2'] = np.abs(df['Ch2'])
df['Ch3'] = np.abs(df['Ch3'])
df['Ch4'] = np.abs(df['Ch4'])

# Assuming the preprocessing steps for the new data are in a function preprocess_data
# Step 3: Preprocess the Data (Assuming df contains the input features)
# Apply the same preprocessing steps that were used during training
# df = preprocess_data(df)

# Step 1: Load the Trained Model
model = joblib.load('random_forest_classifier_model.pkl')  # Replace with your model file path

# Step 4: Make Predictions
predictions = model.predict(df)  # Assuming df contains the features used during training

# Step 5: Final Prediction
final_output = int(mode(predictions)[0])
print("Final Output:", final_output)

Final Output: 1


In [6]:
# Import library and create instance of REST client.
from Adafruit_IO import Client, Data

aio = Client("angeln", "aio_cztB48BdocY2se5Ki6gfxFCR3oPY")

# Add the value to the feed.
new_data = aio.feeds('boreme-data')
aio.send_data(new_data.key, final_output)

ModuleNotFoundError: No module named 'Adafruit_IO'